In [ ]:
import random
import string
import pickle
import json
import subprocess 

In [ ]:
def random_primitive():
    return random.choice([
        random.randint(-1000, 1000),
        random.uniform(-1000, 1000),
        ''.join(random.choices(string.ascii_letters + string.digits, k=random.randint(1, 20))),
        random.choice([True, False, None])
    ])

def generate_random_object(depth=0, max_depth=4):
    if depth >= max_depth:
        return random_primitive()

    obj_type = random.choice(['list', 'dict', 'tuple', 'set', 'primitive'])

    if obj_type == 'list':
        return [generate_random_object(depth + 1, max_depth) for _ in range(random.randint(0, 5))]
    elif obj_type == 'dict':
        return {
            generate_random_object(depth + 1, max_depth): generate_random_object(depth + 1, max_depth)
            for _ in range(random.randint(0, 5))
        }
    elif obj_type == 'tuple':
        return tuple(generate_random_object(depth + 1, max_depth) for _ in range(random.randint(0, 5)))
    elif obj_type == 'set':
        return set(generate_random_object(depth + 1, max_depth) for _ in range(random.randint(0, 5)))
    else:
        return random_primitive()

In [ ]:
def pickle_roundtrip(obj, protocol=4):
    try:
        data = pickle.dumps(obj, protocol=protocol)
        return pickle.loads(data)
    except Exception as e:
        return f"ERROR: {e}"

def run_pickle_fuzz_tests(num_tests=100, max_depth=4):
    results = []
    for _ in range(num_tests):
        obj = generate_random_object(max_depth=max_depth)
        roundtrip = pickle_roundtrip(obj, protocol=4)
        results.append((obj, roundtrip))
    return results

In [ ]:
with open("pickle_fuzz_runner.py", "w") as f:
    f.write('''import json
import pickle
import random
import string

def random_primitive():
    return random.choice([
        random.randint(-1000, 1000),
        random.uniform(-1000, 1000),
        ''.join(random.choices(string.ascii_letters + string.digits, k=random.randint(1, 20))),
        random.choice([True, False, None])
    ])

def generate_random_object(depth=0, max_depth=4):
    if depth >= max_depth:
        return random_primitive()

    obj_type = random.choice(['list', 'dict', 'tuple', 'set', 'primitive'])

    if obj_type == 'list':
        return [generate_random_object(depth + 1, max_depth) for _ in range(random.randint(0, 5))]
    elif obj_type == 'dict':
        return {
            generate_random_object(depth + 1, max_depth): generate_random_object(depth + 1, max_depth)
            for _ in range(random.randint(0, 5))
        }
    elif obj_type == 'tuple':
        return tuple(generate_random_object(depth + 1, max_depth) for _ in range(random.randint(0, 5)))
    elif obj_type == 'set':
        return set(generate_random_object(depth + 1, max_depth) for _ in range(random.randint(0, 5)))
    else:
        return random_primitive()

def pickle_roundtrip(obj, protocol=4):
    try:
        data = pickle.dumps(obj, protocol=protocol)
        return pickle.loads(data)
    except Exception as e:
        return f"ERROR: {e}"

def run_pickle_fuzz_tests(num_tests=100, max_depth=4):
    results = []
    for _ in range(num_tests):
        obj = generate_random_object(max_depth=max_depth)
        roundtrip = pickle_roundtrip(obj, protocol=4)
        results.append((str(obj), str(roundtrip)))
    return results

if __name__ == "__main__":
    res = run_pickle_fuzz_tests()
    json.dump(res, fp=open("/dev/stdout", "w"))
''')

In [ ]:
def run_with_python_version(python_cmd):
    try:
        result = subprocess.run(
            [python_cmd, 'pickle_fuzz_runner.py'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            check=True,
            text=True
        )
        return json.loads(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"[{python_cmd}] Error:\n", e.stderr)
        return None

versions = {
    'py37': 'python3.7',
    'py38': 'python3.8',
    'py39': 'python3.9'
}

all_results = {}
for label, cmd in versions.items():
    print(f"Running fuzz test using {cmd}...")
    res = run_with_python_version(cmd)
    all_results[label] = res

In [ ]:
print("\n🔍 Difference Report:")
for i in range(100):
    results = [all_results[v][i][1] if all_results[v] else "ERR" for v in versions]
    if len(set(results)) != 1:
        print(f"\n[MISMATCH #{i}]")
        for v in versions:
            print(f"{v}: {results[list(versions.keys()).index(v)]}")